# Test monthly average

First make up some data and calculate the averages, then try using my function, and see you get the same answer.

In [151]:
#import my functions
import sys
sys.path.append('../')
import helpers.fileHandler as fh
import utils.sstIndex as sst
import utils.pslIndex as psl
import utils._indexDefinitions as _index
import averageForTimePeriod from utils/timePeriod

In [152]:
import xarray
import numpy
import cftime
import pandas

I am going to make up some data, and put it in a pandas array first

In [153]:
#first make up some time period
dateRange=xarray.cftime_range(
    start=cftime.DatetimeNoLeap(1995,1,1),
    end=cftime.DatetimeNoLeap(1999,12,1),
    freq='M')

In [155]:
#put it into a pd Dataframe
testPd=pandas.DataFrame(index=dateRange)

testPd.index.name='time'

In [164]:
#add some rando data
for keys in _index.monthsOfInterest:
    testPd[keys]=5*numpy.random.rand(len(dateRange),1)

I am now going to calculate the averages in a really easy to understand way

In [169]:
#years of interest
expectedAv=pandas.DataFrame(index=[1995,1996,1997,1998])
expectedAv.index.name='year'

#for every index defined in the index file
for keys in ['nino12', 'nino3', 'nino34', 'nino4', 'westIO', 'eastIO', 'indian_ocean_dipole', 'sam']:
    answer = list()
    
    #grab the time period defined in the index file
    #this line is a bit lazy, i really should define these indeces some other way incase of systematic errors
    monthPeriod=_index.monthsOfInterest[keys].copy()

    #select the data for this index
    indexArray = testPd[keys]
    
    temp = 0
    #for every month in the data
    for monthIter in numpy.arange(0,len(indexArray),1):
        # a month in the time period of interest:
        if ((monthIter+1)>=monthPeriod[0] and (monthIter+1)<=monthPeriod[1]):
            temp += indexArray.values[monthIter]
        else:
        # not a month in the time period of interest
            if temp!=0:
                answer.append(temp/(monthPeriod[1]-monthPeriod[0]+1))
                temp = 0
                monthPeriod[0]+=12
                monthPeriod[1]+=12
    #save output for this index and move to the next one
    expectedAv[keys]=answer


expectedAv.round(5)

,nino12,nino3,nino34,nino4,westIO,eastIO,indian_ocean_dipole,sam
year,,,,,,,,
1995,2.53997,2.72517,2.95969,2.52710,2.48979,2.10115,2.85667,3.01686
1996,2.76566,2.63620,2.82071,2.21208,1.97232,2.99854,2.69213,2.53692
1997,2.65205,1.16205,3.03298,3.26332,1.14362,2.99186,2.72766,2.25631
1998,2.55639,2.73185,2.29786,2.47164,3.11586,3.81947,1.83710,3.77692


Calculate it using the function

In [170]:
#first cast it into an xarray
testXr = xarray.Dataset(testPd)

# do the calculation
calculatedAvs=averageForTimePeriod(testXr)
calculatedAvs.round(5)

,nino12,nino3,nino34,nino4,westIO,eastIO,indian_ocean_dipole,sam
year,,,,,,,,
1995,2.53997,2.72517,2.95969,2.52710,2.48979,2.10115,2.85667,3.01686
1996,2.76566,2.63620,2.82071,2.21208,1.97232,2.99854,2.69213,2.53692
1997,2.65205,1.16205,3.03298,3.26332,1.14362,2.99186,2.72766,2.25631
1998,2.55639,2.73185,2.29786,2.47164,3.11586,3.81947,1.83710,3.77692


In [171]:
calculatedAvs==expectedAv

,nino12,nino3,nino34,nino4,westIO,eastIO,indian_ocean_dipole,sam
year,,,,,,,,
1995,False,False,True,False,True,False,False,True
1996,False,True,True,False,True,True,True,True
1997,False,True,False,True,True,True,True,True
1998,False,True,True,True,True,True,True,True


These values are the same, but somewhere in there the last digit (of a float64) ended up different. Not overly significant.